In [1]:
import importlib
import torch
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import torchvision.transforms as transforms
import os
from data import BraTSDataset
import torchvision.utils as vutils
from PIL import Image

In [2]:
from gan.UNetGenerator import UNetGenerator
import config.unetConfig as cfg

In [2]:
image_size = 64

### Store T1/T1ce real images

In [3]:
dataset_root = "dataset"
t1_train_data = "data/MICCAI_BraTS2020/train/t1ce"
store_path = "images/t1ce/real"
image_paths = [
    os.path.join(t1_train_data, impath) for impath in os.listdir(t1_train_data)
]
batch_size = len(image_paths)
tf = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    ])
dataset = BraTSDataset(image_paths, tf)

In [ ]:
dataset.save(store_path)

In [4]:
# Model image mapping

# wgan_18_35_30_11       -> wgan_gp_t1
# psnr_threshold = 17.5
# ssim_threshold = 0.65

# wgan_t1ce_21_21_30_11  -> wgan_gp_t1ce
# psnr_threshold = 17.5
# ssim_threshold = 0.65

# wgan_t1ce_22_14_30_11  -> wgan_t1ce
# psnr_threshold = 14.5
# ssim_threshold = 0.35

# wgan_20_22_29_11       -> wgan_t1
# psnr_threshold = 14.5
# ssim_threshold = 0.35

# dcgan_00_07_01_12      -> dcgan_t1
# psnr_threshold = 16.5
# ssim_threshold = 0.55

# dcgan_t1ce_08_17_02_12 -> dcgan_t1ce
# psnr_threshold = 18.5
# ssim_threshold = 0.55

# unetgan/norm_winit/unetgan_T1_Gstate__15epochs_64batch_G_300z_64feat_0.001lr_D_4lvl_64feat_0.0002lr_0.2lRelu -> images/t1/unetgan_norm_winit

# unetgan/norm_winit/unetgan_T1CE_Gstate__15epochs_64batch_G_300z_64feat_0.001lr_D_4lvl_64feat_0.0002lr_0.2lRelu -> images/t1ce/unetgan_norm_winit

# unetgan/ortho_winit/unetgan_T1_Gstate__15epochs_64batch_G_300z_64feat_0.001lr_D_4lvl_64feat_0.0002lr_0.2lRelu -> images/t1/unetgan_ortho_winit

# unetgan/ortho_winit/unetgan_T1CE_Gstate__15epochs_64batch_G_300z_64feat_0.001lr_D_4lvl_64feat_0.0002lr_0.2lRelu -> images/t1ce/unetgan_ortho_winit

model_folder = "models/dcgan_t1ce_08_17_02_12"
image_folder = "images/t1ce/dcgan"
# psnr_threshold = 18.5
# ssim_threshold = 0.55

In [5]:
def compute_psnr(real_batch: np.ndarray, fake_batch: np.ndarray) -> float:
    b_size = real_batch.shape[0]
    psnr_val = 0.0
    # fake_batch will only have one batch
    for i in range(b_size):
        psnr_val += psnr(
            real_batch[i, :, :, :].transpose(1, 2, 0),
            fake_batch[0, :, :, :].transpose(1, 2, 0),
            data_range=1.0,
        )
    return psnr_val / b_size

In [6]:
def compute_ssim(real_batch: np.ndarray, fake_batch: np.ndarray) -> float:
    b_size = real_batch.shape[0]
    ssim_val = 0.0
    # fake_batch will only have one batch
    for i in range(b_size):
        ssim_val += ssim(
            real_batch[i, :, :, :],
            fake_batch[0, :, :, :],
            channel_axis=0,
            data_range=1.0,
        )
    return ssim_val / b_size

In [7]:
model_library = ".".join(model_folder.split("/"))
model = importlib.import_module(f"{model_library}.model")

In [8]:
model_g = model.Generator().to(model.device)
model_g.load_state_dict(torch.load(f"{model_folder}/generator.pth"))
model_g.eval()
print(model_g)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(32, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [8]:
netG = UNetGenerator(cfg.LATENT_SZ, cfg.NGF, cfg.NGC).to(cfg.DEVICE)
netG.load_state_dict(torch.load(f"{model_folder}.pth"))
netG.eval()
print(netG)

UNetGenerator(
  (main): Sequential(
    (0): ConvTranspose2d(300, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01, inplace=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01, inplace=True)
    (1

In [ ]:
tf = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    ])

num_iter_per_image = 100
for i in range(len(image_paths)):
    real = dataset.__getitem__(i).numpy()[None, ...]
    best_psnr_val = float("-inf")
    best_ssim_val = None
    best_image = None
    for j in range(num_iter_per_image):

        noise = torch.randn(1, cfg.LATENT_SZ, 1, 1, device=cfg.DEVICE)
        fake_image = netG(noise).detach().cpu().numpy()
        fake_image = fake_image[0][0]
        fake_image = np.uint8(fake_image * 255)
        fake_image = Image.fromarray(fake_image, mode="L")
        fake_image = tf(fake_image)
        fake_image = fake_image.numpy()[None, ...]

        psnr_val = compute_psnr(real, fake_image)
        if psnr_val > best_psnr_val:
            best_psnr_val = psnr_val
            best_image = fake_image.copy()
            best_ssim_val = compute_ssim(real, fake_image)
    print(f"Best image PSNR: {best_psnr_val}")
    print(f"Best image SSIM: {best_ssim_val}")
    best_image = torch.Tensor(best_image)
    vutils.save_image(best_image, f"{image_folder}/{i}.png")
    print(f"Saved {i+1} image")

In [9]:
num_iter_per_image = 100
for i in range(len(image_paths)):
    best_psnr_val = float("-inf")
    best_ssim_val = None
    best_image = None
    real = dataset.__getitem__(i).numpy()[None, ...]
    for j in range(num_iter_per_image):
        noise = torch.randn(1, model.latent_size, 1, 1, device=model.device)
        fake_image = model_g(noise).detach().cpu().numpy()
        psnr_val = compute_psnr(real, fake_image)
        if psnr_val > best_psnr_val:
            best_psnr_val = psnr_val
            best_image = fake_image
            best_ssim_val = compute_ssim(real, fake_image)
    print(f"Best image PSNR: {best_psnr_val}")
    print(f"Best image SSIM: {best_ssim_val}")
    best_image = torch.Tensor(best_image)
    vutils.save_image(best_image, f"{image_folder}/{i}.png")
    print(f"Saved {i+1} image")

MIOpen(HIP): Warning [SQLiteBase] Missing system database file: gfx1030_14.kdb Performance may degrade. Please follow instructions to install: https://github.com/ROCmSoftwarePlatform/MIOpen#installing-miopen-kernels-package


Best image PSNR: 25.65971089209464
Best image SSIM: 0.7312260270118713
Saved 1 image
Best image PSNR: 25.41623112668509
Best image SSIM: 0.6713578701019287
Saved 2 image
Best image PSNR: 29.49515381041218
Best image SSIM: 0.730783224105835
Saved 3 image
Best image PSNR: 26.066124500832682
Best image SSIM: 0.6847184300422668
Saved 4 image
Best image PSNR: 29.34069534694914
Best image SSIM: 0.7471868395805359
Saved 5 image
Best image PSNR: 23.71843114205017
Best image SSIM: 0.714297890663147
Saved 6 image
Best image PSNR: 27.02925004177135
Best image SSIM: 0.7294492125511169
Saved 7 image
Best image PSNR: 27.43956508402576
Best image SSIM: 0.7644181847572327
Saved 8 image
Best image PSNR: 26.61172449410469
Best image SSIM: 0.7269890308380127
Saved 9 image
Best image PSNR: 29.0806211157812
Best image SSIM: 0.7681668996810913
Saved 10 image
Best image PSNR: 26.000678894522963
Best image SSIM: 0.6618914604187012
Saved 11 image
Best image PSNR: 28.331159246981752
Best image SSIM: 0.735203325

In [ ]:
# fake = np.zeros((batch_size, 1, model.image_size, model.image_size))
# # Don't stop until required number of images have been saved
# total_num_images = batch_size
# num_images = 0
# while(True):
#     noise = torch.randn(1, model.latent_size, 1, 1, device=model.device)
#     fake_image = model_g(noise).detach().cpu().numpy()
#     psnr_val = compute_psnr(real, fake_image)
#     ssim_val = compute_ssim(real, fake_image)
#     if psnr_val > psnr_threshold and ssim_val > ssim_threshold:
#         fake[num_images] = fake_image
#         num_images += 1
#         print(f"Saved image with psnr: {psnr_val}, ssim: {ssim_val}, total: {num_images}")
#         # Save fake_image
#         # Stop if we have generated equal number of fake images
#         # as compared to real images
#         if num_images >= total_num_images:
#             break

In [ ]:
# real = torch.Tensor(real)
# fake = torch.Tensor(fake)
# num_images = real.shape[0]
# print(f"Saving {num_images} images")
# # Save images
# for i in range(num_images):
#     # Save real
#     vutils.save_image(real[i], f"{image_folder}/real/{i+1}.png")

#     # Save fake
#     vutils.save_image(fake[i], f"{image_folder}/fake/{i+1}.png")